# <div style='text-align: center;'> **Projet Gaming** - Partie 1 : Extraction des données via les API *IGDB*, API du *Steam Store*, *Steam Web API* et *SteamSpy* </div>

## Introduction

Ce projet consiste en une série d'analyses appliquées à un jeu de données concernant l'**industrie du jeu vidéo**. 
L'objectif principal est de mettre en pratique des concepts de data science en Python et de démontrer ma capacité à suivre les évolutions technologiques ; il s'agit également pour moi de développer mes compétences dans la gestion de problématiques métier, afin d'élargir ma culture professionnelle tout en produisant des insights pertinents pour l'industrie du jeu vidéo.

Ici, je me suis focalisée sur une catégorie de jeux : **les jeux RGP** (*Role-Playing Games* ou *jeux de rôle* en français), un genre de jeu où les joueurs prennent le contrôle de personnages fictifs dans un monde virtuel.

Afin de travailler ma capacité à manipuler des **données provenant de sources externes**, j'ai décidé de collecter mes données depuis différentes **API** : l'API *IGDB (Twitch)*, l'API du *Steam Store*, *Steam Web API* et enfin *SteamSpy*. Ces interfaces permettent d'extraire des informations détaillées sur l'industrie du jeu vidéo relatives à différents aspects du marché, tels que les comportements d'achat, les préférences des joueurs, les performances commerciales des jeux, etc.  
**La première phase de ce projet se concentre sur l'extraction de ces données.**  

Les étapes suivantes portent sur l’analyse de ces données à travers une série de questions qui seront détaillées au fur et à mesure du projet. A titre d'exemple, on y retrouve :
- **Analyses descriptives** : corrélation entre les prix et les dates de sortie, les plateformes et les modes de jeu.
- **Analyses prédictives** (approche statistique + approche Machine Learning) : évaluation de l'impact de l'année de sortie, de la note, du prix et du nombre de recommandations positives sur la popularité des jeux.
- **Clustering** et **segmentation** sur des données textuelles **(NLP avec scoring)** : identification de groupes de jeux similaires en fonction de leurs mécaniques, analysées à partir des résumés.

## Importation des bibliothèques


In [3]:
import requests      # Pour effectuer des requêtes HTTP
import datetime      # Pour manipuler les dates
import pandas as pd  # Pour manipuler les données tabulaires
import os            # Pour gérer les chemins de fichiers
import time          # Pour introduire des délais entre les requêtes
from bs4 import BeautifulSoup  # Pour parser et extraire des données à partir de pages HTML
import json          # Pour lire et écrire des fichiers JSON
from urllib.parse import quote  # Pour encoder des chaînes de texte dans une URL

## Importation des données

### Métadonnées issue de l'API *IGDB (Twitch)*
L’API ***IGDB** (Internet Game Database)*, accessible *via* *Twitch*, est une base de données extrêmement complète sur les jeux vidéo. Elle permet d’extraire un ensemble de métadonnées (données descriptives) très utiles sur les jeux.  

Pour y accéder, il est nécessaire d'avoir un compte *Twitch Developer* pour obtenir un client_id et un access_token. Les requêtes se font en POST – et non en GET, pourtant plus courant pour la lecture de données dans les API de type REST telles que l'API *IGDB*. Ce choix s’explique par la syntaxe spécifique de l’API *IGDB* (nommée *Apicalypse*, proche du SQL), qui nécessite souvent des requêtes complexes. Le corps de la requête POST permet de structurer cette complexité, ce qui ne serait pas possible avec une requête GET classique.

#### Authentification à l'API

In [4]:
CLIENT_ID = "hkmuvrxev3jgb6077y6v5b719ot319"
ACCESS_TOKEN = "fq9tkosl71ghep16hnjdfietime65g"

# En-têtes HTTP pour l'authentification auprès de l'API
headers = {
    "Client-ID": CLIENT_ID,
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

#### Requête pour l'extraction des données de l'API

Ici, j'extrais une série de variables **pour les 75 jeux RPG les mieux notés** : 

- Le nom du jeu vidéo ;
- Le résumé ou la description du jeu ;
- La date de première sortie du jeu ;
- Les noms des plateformes sur lesquelles le jeu est disponible ;
- La note agrégée du jeu (moyenne des critiques professionnelles) ;
- Les noms des entreprises impliquées dans le développement/édition du jeu ;
- Les différents modes de jeu disponibles (solo, multijoueur, etc.).

L'API *IGDB* fournit les **métadonnées** des jeux mais ne donne pas de **statistiques relatives au marché**.
Les métadonnées peuvent être mixées avec les données statistiques d'autres API pour fournir des détails commerciaux, des informations relatives au comportement des joueurs, etc.
Dans cette étude, nous bornons les analyses à une partie seulement des jeux RPG extraits de l'API *IGDB*, **ceux disponibles sur *Steam***. *Steam* est l'une des plus importantes plateformes pour acheter, télécharger et jouer à des jeux vidéo en ligne développée par *Valve Corporation*, avec des millions d'utilisateurs actifs. Les données étudiées ne concernent donc pas les jeux disponibles sur console ou mobile.

En plus des variables mentionnées précédemment, l'API *IGDB* fournit des informations d'identification pour les jeux sur diverses plateformes, dont *Steam*.  
J'effectue donc ici en complément une recherche de l'**AppID *Steam*** (à travers l'endpoint external_games de l'API), un identifiant essentiel qui servira de "clé primaire" pour toutes mes requêtes ultérieures sur les API *Steam*. Cet identifiant me permettra d'extraire les données avec plus de fiabilité qu'en utilisant le nom du jeu, qui peut varier selon les plateformes.

In [5]:
url = "https://api.igdb.com/v4/games" # URL de l'endpoint des jeux de l'API IGDB

# Corps de la requête en syntaxe Apicalypse
# N.B. : Apicalypse est un langage très simplifié de requête développé par les créateurs d’IGDB pour interroger facilement leur base de données
body = """
fields name, summary, first_release_date, platforms.name, aggregated_rating,
involved_companies.company.name, game_modes.name;
where genres = 12 & aggregated_rating != null;
sort aggregated_rating desc;
limit 75;
"""
# where genres = 12 : filtre les jeux du genre 12 (qui correspond aux RPG dans la taxonomie IGDB)

# Envoi de la requête POST à l'API
# N.B. : POST permet d'envoyer des corps de requête plus longues et plus complexes que GET ; IGDB a conçu son API pour utiliser POST comme méthode standard de requêtage
response = requests.post(url, headers=headers, data=body)

# Liste qui contiendra les données de jeux formatées
games_data = []

# Vérification que la requête a réussi (code 200)
if response.status_code == 200:
    data = response.json() # Conversion de la réponse JSON en liste de dictionnaires Python

    for game in data:
        game_info = {} # Dictionnaire pour stocker les informations du jeu actuel

        game_info["Nom"] = game['name']
        game_info["Résumé"] = game.get('summary', 'Non disponible')

        # Traitement de la date de sortie
        release_date = game.get('first_release_date')
        if release_date:
            # Conversion du timestamp Unix en format de date lisible
            release_date = datetime.datetime.fromtimestamp(release_date, datetime.UTC).strftime('%Y-%m-%d')
        else:
            release_date = "Non disponible"
        game_info["Date de sortie"] = release_date

        game_info["Note"] = round(game.get('aggregated_rating'), 2)

        # Extraction et formatage des plateformes disponibles
        platforms = [p['name'] for p in game.get('platforms', [])]
        game_info["Plateformes"] = ', '.join(platforms) if platforms else 'Non disponible'

        # Extraction et formatage des entreprises impliquées (développeurs/éditeurs)
        companies = [c['company']['name'] for c in game.get('involved_companies', [])]
        game_info["Développeurs/Éditeurs"] = ', '.join(companies) if companies else 'Non disponible'

        # Extraction et formatage des modes de jeu
        game_modes = [m['name'] for m in game.get('game_modes', [])]
        game_info["Modes de jeu"] = ', '.join(game_modes) if game_modes else 'Non disponible'

        # Recherche de l'AppID Steam via l'endpoint external_games
        external_url = "https://api.igdb.com/v4/external_games"
        external_body = f"""
        fields uid, category;
        where category = 1 & game = {game['id']};
        """
        ext_response = requests.post(external_url, headers=headers, data=external_body)
        if ext_response.status_code == 200 and ext_response.json():
            steam_data = ext_response.json()[0]
            game_info["AppID Steam"] = steam_data.get("uid", "Non disponible")
        else:
            game_info["AppID Steam"] = "Non disponible"

        # Ajout des informations du jeu à la liste principale
        games_data.append(game_info)

        # Pause pour éviter un rate limit 
        time.sleep(0.2)

    # Création d'un dataframe à partir de la liste de dictionnaires
    df_IGDB = pd.DataFrame(games_data)

    # Utilisation du répertoire courant pour enregistrer le fichier et exportation du df au format csv
    csv_path = "IGDB_jeux_rpg.csv"
    df_IGDB.to_csv(csv_path, index=False, encoding='utf-8-sig')

    print(f"Les données ont été enregistrées dans '{os.path.abspath(csv_path)}'")
    display(df_IGDB.head())
else:
    print(f"Erreur {response.status_code} : {response.text}")


Les données ont été enregistrées dans 'C:\Users\leaam\IGDB_jeux_rpg.csv'


,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu,AppID Steam
0,EverQuest: Evolution,contains the original EverQuest game with all ...,2003-08-25,100.00,PC (Microsoft Windows),Non disponible,Massively Multiplayer Online (MMO),Non disponible
1,Dark Age of Camelot,Dark Age of Camelot is a 3D medieval fantasy M...,2001-10-10,100.00,PC (Microsoft Windows),Mythic Entertainment,Massively Multiplayer Online (MMO),Non disponible
2,Dragon Age: Origins Collector's Edition,Dragon Age: Origins Collector's Edition contai...,2009-12-03,98.00,PlayStation 3,"Electronic Arts, BioWare",Single player,Non disponible
3,World of Warcraft: Wrath of the Lich King,"World of Warcraft: Wrath of the Lich King, oft...",2008-11-13,96.67,"PC (Microsoft Windows), Mac",Blizzard Entertainment,Massively Multiplayer Online (MMO),Non disponible
4,Baldur's Gate: Siege of Dragonspear - Collecto...,The Collector's Edition features a limited-edi...,2016-03-31,95.00,"PC (Microsoft Windows), Mac","Overhaul Games, Beamdog",Single player,Non disponible


In [7]:
# Liste des jeux pour lesquels l'API n'a pas renvoyé d'AppID
print(df_IGDB[df_IGDB["AppID Steam"] == "Non disponible"]["Nom"])

Pour chacun des jeux de cette liste, soit l'API ne renvoie pas d'AppID car **le jeu n'est pas présent sur *Steam***, soit **il est présent** mais l'AppID n'est pas renseigné sur *IGDB* ou est associé à un autre nom correspondant au même jeu mais avec une syntaxe légèrement différente.  
La recherche de correspondance nominale est une tâche laborieuse : les critères de correspondance que l'on choisit sont souvent soit trop stricts, soit trop permissifs. Pour cette raison, j'ai décidé d'effectuer une recherche manuelle directement depuis *Steam* pour (1) déterminer si le jeu est présent sur la plateforme et (2) si oui, récupérer son AppID (quand on ouvre la page d’un jeu sur *Steam*, l’AppID est dans l’URL).

In [9]:
# Dictionnaire avec les AppID trouvés manuellement (quand confusion possible sur le nom et/ou choix fait d'analyser une version étendue du jeu incluant l'extension ou le DLC d'intérêt, un commentaire le mentionne)
appid_corrections = {
    "Dragon Age: Origins Collector's Edition": 17450, # Disponible sur Steam sous le nom Dragon Age: Origins
    "Baldur's Gate: Siege of Dragonspear - Collecto...": 228280, # Disponible sur Steam sous le nom Baldur's Gate: Siege of Dragonspear
    "Atelier Meruru: The Apprentice of Arland": 936190,
    "Dark Cloud 2": 587590, # Disponible sur Steam sous le nom Dark Chronicle
    "Baldur's Gate II: Shadows of Amn": 257350,
    "Odin Sphere: Leifthrasir": 527230, # Inclus sur Steam dans Odin Sphere, que l'on prendra dans sa globalité pour l'étude
    "Final Fantasy XIV: Heavensward": 39210, # Inclus sur Steam dans Final Fantasy XIV Online (extensions disponibles en achat intégré) qu'on prendra également dans sa globalité
    "El Shaddai: Ascension of the Metatron": 1292630,
    "Grim Dawn: Definitive Edition": 219990, # Inclus sur Steam dans Grim Dawn avec des DLCs correspondant à la Definitive Edition, qu'on prendra dans sa globalité
    "Atelier Mysterious Trilogy DX": 1446650,  
    "Hyperdimension Neptunia Victory": 282900, # Disponible sur Steam sous le nom Hyperdimension Neptunia Re;Birth3 V Generation
    "Guild Wars 2: Heart of Thorns": 1284210, # Inclus sur Steam dans Guild Wars 2 (extensions disponibles en achat intégré) qu'on prendra dans sa globalité
    "Romancing SaGa": 1341200, # Plusieurs versions sur Steam, choix de prendre Romancing SaGa 3
    "Mega Man Battle Network 3 Blue": 1798010, # Inclus sur Steam dans Mega Man Battle Network Legacy Collection qu'on prendre dans sa globalité
    "Sacred": 12320, # Inclus sur Steam dans Sacred Gold (édition complète incluant le jeu de base - Sacred - et l'extension Sacred Plus) qu'on prendre dans sa globalité)
    "Cladun: This is an RPG": 446540,
    "Guild Wars 2: Path of Fire": 1284210, # Inclus sur Steam dans Guild Wars 2 (extensions disponibles en achat intégré) qu'on prendra dans sa globalité => Attention, redondance avec "Guild Wars 2: Heart of Thorns" (nous les analyserons ensemble pour la suite)
    "Shiren the Wanderer: The Tower of Fortune and ...": 1178790,
    "Kingdom Hearts Birth by Sleep": 2552430, # Inclus sur Steam dans la collection KINGDOM HEARTS -HD 1.5+2.5 ReMIX- qu'on prendra dans sa globalité
    "Void Terrarium++": 1726670
}

# Mise à jour des AppID dans le df
for game_name, app_id in appid_corrections.items():
    df_IGDB.loc[df_IGDB['Nom'] == game_name, 'AppID Steam'] = app_id

# Suppression du jeu "Guild Wars 2: Heart of Thorns" et renommage du jeu "Guild Wars 2: Path of Fire" en "Guild Wars 2" 
df_IGDB = df_IGDB[df_IGDB['Nom'] != "Guild Wars 2: Heart of Thorns"]
df_IGDB['Nom'] = df_IGDB['Nom'].replace("Guild Wars 2: Path of Fire", "Guild Wars 2")

# Vérification : affichage des lignes modifiées
# print(df_IGDB[df_IGDB['Nom'].isin(appid_corrections.keys())][['Nom', 'AppID Steam']])

# Sauvegarde du fichier modifié
df_IGDB.to_csv(csv_path, index=False, encoding='utf-8-sig')

,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu,AppID Steam
0,EverQuest: Evolution,contains the original EverQuest game with all ...,2003-08-25,100.00,PC (Microsoft Windows),Non disponible,Massively Multiplayer Online (MMO),Non disponible
1,Dark Age of Camelot,Dark Age of Camelot is a 3D medieval fantasy M...,2001-10-10,100.00,PC (Microsoft Windows),Mythic Entertainment,Massively Multiplayer Online (MMO),Non disponible
2,Dragon Age: Origins Collector's Edition,Dragon Age: Origins Collector's Edition contai...,2009-12-03,98.00,PlayStation 3,"Electronic Arts, BioWare",Single player,17450
3,World of Warcraft: Wrath of the Lich King,"World of Warcraft: Wrath of the Lich King, oft...",2008-11-13,96.67,"PC (Microsoft Windows), Mac",Blizzard Entertainment,Massively Multiplayer Online (MMO),Non disponible
4,Baldur's Gate: Siege of Dragonspear - Collecto...,The Collector's Edition features a limited-edi...,2016-03-31,95.00,"PC (Microsoft Windows), Mac","Overhaul Games, Beamdog",Single player,Non disponible
...,...,...,...,...,...,...,...,...
70,World of Warcraft: Shadowlands,The veil between life and death is no more.\nW...,2020-11-23,82.40,"PC (Microsoft Windows), Mac",Blizzard Entertainment,Massively Multiplayer Online (MMO),Non disponible
71,Persona Q: Shadow of the Labyrinth,"Persona Q is a crossover video game, containin...",2014-06-05,82.25,Nintendo 3DS,"NIS America, Atlus, P Studio, Atlus USA, Atlus...",Single player,Non disponible
72,Tales of Berseria,In Tales Of Berseria players embark on a journ...,2016-08-18,82.14,"PlayStation 3, PlayStation 4, PC (Microsoft Wi...","Bandai Namco Entertainment Europe, Bandai Namc...","Single player, Multiplayer, Co-operative",429660
73,Xenoblade Chronicles X: Definitive Edition,The year is 2054. Earth has been destroyed by ...,2025-03-20,82.00,Nintendo Switch,"Nintendo, Monolith Soft","Single player, Multiplayer",Non disponible


In [10]:
# Création d'une liste avec uniquement les jeux présents sur Steam et leur AppID
# Filtrage pour garder uniquement les jeux avec un AppID Steam valide
jeux_steam = df_IGDB[df_IGDB['AppID Steam'] != "Non disponible"]

# Sélection des colonnes nécessaires uniquement
jeux_steam = jeux_steam[['Nom', 'AppID Steam']]

# Transformation en format JSON et sauvegarde
jeux_steam.to_json('steam_jeux_liste.json', orient='records')

# Vérification
print(f"Nombre de jeux avec un AppID Steam valide: {len(jeux_steam)}")
print(jeux_steam)

Nombre de jeux avec un AppID Steam valide: 44
                                                  Nom AppID Steam
2             Dragon Age: Origins Collector's Edition       17450
6                                   Final Fantasy XVI     2515020
8                                   Dragon's Dogma II     2054970
9                   Final Fantasy XIV: Shadowbringers     1016870
10                                Dragon Age: Origins       17450
15           Atelier Meruru: The Apprentice of Arland      936190
17                                       Dark Cloud 2      587590
18                   Baldur's Gate II: Shadows of Amn      257350
19                 Warhammer Online: Age of Reckoning       17420
22                           Odin Sphere: Leifthrasir      527230
23                       Final Fantasy XIV: Endwalker     1592500
26                The Elder Scrolls Online: Blackwood     1400970
27                                      Albion Online      761890
28                           T

Parmi les 75 jeux RPG les mieux notés, seulement **44** sont disponibles sur *Steam* : **ils constitueront l'échantillon conservé pour la suite.**

*Attention :* Certains jeux disponibles sur *Steam* ont une partie de leur base de joueurs en dehors de la plateforme (launcher propriétaire). Pour ces jeux-là, les chiffres *Steam* ne représentent donc qu'une fraction de la population totale, mais nous nous concentrerons ici sur l'activité de cette plateforme.

### Base de données statistiques issue des API du *Steam Store*, *Steam Web API* (API officielle) et *SteamSpy*

#### Extraction des données statistiques via 3 API différentes
Afin d'accéder à une diversité de **données statistiques** relatives aux jeux étudiés, je fais appel à 3 API : 
- L'**API du *Steam Store*** nous permet d'obtenir les détails commerciaux et descriptifs des jeux. J'extrais ici :
    - Le prix actuel ;
    - Le score Metacritic (note des critiques, de 0 - *critiques généralement défavorables* - à 100 - *critiques généralement favorables*)  
Les scores Metacritic sont très influents dans l'industrie du jeu vidéo et peuvent affecter les ventes. Ils sont souvent utilisés comme un indicateur rapide de la qualité globale d'un jeu selon la presse spécialisée, bien que ces scores ne reflètent pas nécessairement l'opinion des joueurs ou votre expérience personnelle avec le jeu.  
    - Le nombre de recommandations positives.  

- Je passe ensuite par la ***Steam Web API*** (officielle) pour déterminer le nombre de joueurs actuels. Cette API est actuellement le seul endpoint public connu qui donne en temps réel le nombre de joueurs connectés à un jeu donné.  

**N.B.** : L'API ne compte que les joueurs connectés à ce moment précis. Les données sont un screen shot d'un instant i, elles varient selon le moment où le script est lancé (*pour information, les dataframes constitués ici sont réalisés à partir d'un screen shot effectué le samedi 13/04/2025 aux alentours de 18h*).  

- Enfin, j'ai recours à l'**API *SteamSpy*** pour récupérer : 
    - Le nombre de propriétaires du jeu ;  
J'effectue aussi une approximation des revenus générés par chaque jeu, en faisant le produit du prix du jeu par le nombre médian de propriétaires du jeu.  

*Attention :* L'API *SteamSpy* fournit des **estimations** relatives au comportement des joueurs pour les jeux disponibles sur *Steam*, et non des chiffres officiels. Elle fonctionne en *scrapant* et en échantillonnant les profils publics *Steam*. Bien que non officielles, cette API donne accès à des informations impossibles à obtenir autrement, d'une grande richesse, rendant son utilisation très utile pour des analyses de tendances, de marché, ou de la data viz.

In [32]:
# Chargement de la liste des jeux depuis le fichier JSON
with open('steam_jeux_liste.json', 'r', encoding='utf-8') as f:
    jeux_steam = json.load(f)
    
# Si jeux_steam est déjà une liste de dictionnaires, on adapte le format
jeux_avec_appid = [
    {"nom": jeu['Nom'], "appid": str(jeu['AppID Steam'])} for jeu in jeux_steam
]

# URLs des APIs
steam_store_url = "https://store.steampowered.com/api/appdetails" # API du Steam Store
steam_stats_url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/" # Steam Web API (officielle)
steamspy_url = "https://steamspy.com/api.php"  # API SteamSpy

# Configuration de headers pour imiter un navigateur (pour éviter les blocages) 
# N.B. : Ces en-têtes, facultatives, sont utilisées pour contourner les restrictions anti-scraping (beaucoup de sites web, dont Steam, ont des mesures pour détecter et bloquer les scripts automatisés qui collectent des données ; ces scripts sont souvent détectés en observant les en-têtes HTTP des requêtes).
# En configurant ces en-têtes, notre script se fait passer pour un vrai navigateur web (ici Chrome) utilisé par un humain.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', # Indique quel navigateur et système d'exploitation on "utilise"
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', # Précise les préférences linguistiques, utile pour avoir des réponses localisées
    'Accept': 'application/json, text/plain, */*', # Définit les types de contenu que le client (i.e. notre script Python) peut traiter (JSON, texte, etc.).
    'Origin': 'https://store.steampowered.com', # Indique d'où provient la requête (certaines API vérifient si les requêtes proviennent de sources légitimes)
    'Referer': 'https://store.steampowered.com/' # Idem
}


# Liste pour stocker les résultats
resultats = []
for jeu in jeux_avec_appid:
    try:
        print(f"Récupération des données pour {jeu['nom']} (AppID: {jeu['appid']})...") 
        
        # Paramètres pour la requête du Store
        store_params = {
            "appids": jeu['appid'],
            "cc": "fr",  # Code pays pour les prix en euros
            "l": "french"  # Langue en français
        }
        
        store_response = requests.get(steam_store_url, params=store_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Paramètres pour les statistiques de joueurs actuels via l'API officielle Steam
        stats_params = {
            "appid": jeu['appid']
        }
        
        stats_response = requests.get(steam_stats_url, params=stats_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Paramètres pour SteamSpy
        steamspy_params = {
            "request": "appdetails",
            "appid": jeu['appid']
        }
        
        steamspy_response = requests.get(steamspy_url, params=steamspy_params)
        time.sleep(1.5)  # Délai pour éviter les limitations
        
        # Initialisation des variables
        prix = "Non disponible"
        metacritic = "N/A"
        recommandations_positives = "N/A"
        joueurs_actuels = "N/A"
        proprietaires = "N/A"
        revenus_approx = "N/A"
        
        # Traitement des données du Store
        if store_response.status_code == 200:
            store_data = store_response.json()
            if store_data and jeu['appid'] in store_data and store_data[jeu['appid']].get('success', False):
                data = store_data[jeu['appid']]['data']
                
                # Prix
                if 'price_overview' in data:
                    prix = f"{data['price_overview'].get('final')/100:.2f} €"
                elif data.get('is_free', False):
                    prix = "Gratuit"
                
                # Score Metacritic
                if 'metacritic' in data:
                    metacritic = data['metacritic'].get('score', "N/A")
                
                # Recommandations positives
                if 'recommendations' in data:
                    recommandations_positives = data['recommendations'].get('total', "N/A")
        
        # Traitement des statistiques de joueurs actuels (API officielle Steam)
        if stats_response.status_code == 200:
            stats_data = stats_response.json()
            if 'response' in stats_data and 'player_count' in stats_data['response']:
                joueurs_actuels = stats_data['response']['player_count']
        
        # Traitement des données SteamSpy
        if steamspy_response.status_code == 200:
            steamspy_data = steamspy_response.json()
                
            # Nombre de propriétaires (format: "min-max")
            if 'owners' in steamspy_data:
                proprietaires = steamspy_data.get('owners', "N/A")

            # Revenus approximatifs (prix * nombre médian de propriétaires)
            if prix != "Non disponible" and prix != "Gratuit" and proprietaires != "N/A":
                try:
                    # Extraction du prix en nombre
                    prix_numerique = float(prix.replace(' €', ''))
                    
                    # Extraction de la médiane des propriétaires
                    if ' .. ' in proprietaires:
                        min_owners, max_owners = map(int, proprietaires.replace(',', '').split(' .. '))
                        median_owners = (min_owners + max_owners) / 2
                        revenus_approx = f"{prix_numerique * median_owners:,.2f} €"
                except:
                    revenus_approx = "Calcul impossible"

        
        # Ajout au résultat
        resultats.append({
            "Nom": jeu['nom'],
            "Prix": prix,
            "Joueurs Actuels": joueurs_actuels,
            "Score Metacritic": metacritic,
            "Recommandations Positives": recommandations_positives,
            "Propriétaires": proprietaires,
            "Revenus Approximatifs": revenus_approx
        })
        
    except Exception as e:
        print(f"Erreur pour {jeu['nom']}: {str(e)}")
        resultats.append({
            "Nom": jeu['nom'],
            "Prix": "Erreur",
            "Joueurs Actuels": "Erreur",
            "Score Metacritic": "Erreur",
            "Recommandations Positives": "Erreur",
            "Propriétaires": "Erreur",
            "Revenus Approximatifs": "Erreur"
        })
        
# Création d'un dataframe avec les résultats
df_jeux_steam = pd.DataFrame(resultats)

# Tri par nombre de joueurs actuels (décroissant)
df_jeux_steam = df_jeux_steam.sort_values(by="Joueurs Actuels", ascending=False)

# Sauvegarde et affichage des résultats
print(f"\nDonnées récupérées pour {len(df_jeux_steam)} jeux")

chemin_sauvegarde = 'Steam_jeux_rpg.csv'
df_jeux_steam.to_csv(chemin_sauvegarde, index=False, encoding='utf-8-sig')
print(f"Données sauvegardées dans {chemin_sauvegarde}")

display(df_jeux_steam)

Récupération des données pour Dragon Age: Origins Collector's Edition (AppID: 17450)...
Récupération des données pour Final Fantasy XVI (AppID: 2515020)...
Récupération des données pour Dragon's Dogma II (AppID: 2054970)...
Récupération des données pour Final Fantasy XIV: Shadowbringers (AppID: 1016870)...
Récupération des données pour Dragon Age: Origins (AppID: 17450)...
Récupération des données pour Atelier Meruru: The Apprentice of Arland (AppID: 936190)...
Récupération des données pour Dark Cloud 2 (AppID: 587590)...
Récupération des données pour Baldur's Gate II: Shadows of Amn (AppID: 257350)...
Récupération des données pour Warhammer Online: Age of Reckoning (AppID: 17420)...
Récupération des données pour Odin Sphere: Leifthrasir (AppID: 527230)...
Récupération des données pour Final Fantasy XIV: Endwalker (AppID: 1592500)...
Récupération des données pour The Elder Scrolls Online: Blackwood (AppID: 1400970)...
Récupération des données pour Albion Online (AppID: 761890)...
Récup

,Nom,Prix,Joueurs Actuels,Score Metacritic,Recommandations Positives,Propriétaires,Revenus Approximatifs
16,Final Fantasy XIV: Heavensward,9.99 €,12649,83,72896,"2,000,000 .. 5,000,000","34,965,000.00 €"
12,Albion Online,Gratuit,6104,N/A,1869,"2,000,000 .. 5,000,000",N/A
32,Star Wars: The Old Republic,Gratuit,2095,85,410,"5,000,000 .. 10,000,000",N/A
35,Guild Wars 2,Gratuit,2052,90,339,"2,000,000 .. 5,000,000",N/A
25,Grim Dawn: Definitive Edition,24.99 €,2037,83,83298,"10,000,000 .. 20,000,000","374,850,000.00 €"
21,Lies of P,59.99 €,972,N/A,37216,"500,000 .. 1,000,000","44,992,500.00 €"
2,Dragon's Dogma II,64.99 €,838,88,69678,"2,000,000 .. 5,000,000","227,465,000.00 €"
1,Final Fantasy XVI,49.99 €,719,N/A,15263,"200,000 .. 500,000","17,496,500.00 €"
40,Kingdom Hearts Birth by Sleep,49.99 €,498,N/A,6257,"500,000 .. 1,000,000","37,492,500.00 €"
17,Suikoden I & II HD Remaster: Gate Rune and Dun...,49.99 €,386,82,1220,"0 .. 20,000","499,900.00 €"


**Remarques** :  
- *Importance des N/A pour le score Metacritic :*  
Metacritic est un site indépendant qui agrège les critiques de jeux vidéo, mais il n’est pas nécessairement associé à tous les jeux sur *Steam*. Certains jeux, notamment ceux indépendants ou plus anciens, n'ont souvent pas été évalués par suffisamment de critiques pour recevoir un score officiel. Metacritic tend à couvrir principalement les titres bénéficiant d'une visibilité médiatique significative, ce qui explique l'absence de scores pour certains jeux.

- *Nombre de joueurs actuels parfois étonnament bas :*   
Des nombres de joueurs actuels quasi nuls voire nuls s'expliquent par l'ancienneté de certains jeux.  
*P.ex.* : Le jeu *Dungeon Siege III: Treasures of the Sun* est paru en 2011. Une vérification manuelle sur *Steam Charts*, site qui recense une série de statistiques sur les jeux disponibles sur *Steam*, permet de confirmer la valeur de 0 joueurs actuels.  

&#8594; **Une vérification manuelle sur *Steam Charts* a été opérée pour vérifier la pertinence de l'output pour tous les jeux pour lesquels le nombre de joueurs actuels trouvé est inférieur à 30.**  

Pour 6 jeux (*Final Fantasy XIV Online, Final Fantasy XIV: Stormblood, The Elder Scrolls Online: Blackwood, Final Fantasy XIV: Endwalker, Warhammer Online: Age of Reckoning et Dark Cloud 2*), les requêtes sur l'API du *Steam Store* et sur *Steam Web API* ont échoué. Ceci peut être lié à divers facteurs, et notamment au fait qu'il peut s'agir de jeux avec lanceurs propriétaires : en effet, la plupart de ces jeux sont des MMO (Final Fantasy XIV et ses extensions, Elder Scrolls Online, Warhammer Online) qui utilisent généralement leurs propres lanceurs et systèmes de compte, même s'ils sont vendus sur *Steam*, ce qui complique le suivi des statistiques. Une autre raison peut être que certains AppID peuvent correspondre à d'anciennes versions, des bundles, ou des versions régionales qui ne sont plus activement suivies par les API.  
J'ai donc décidé d'ôter ces 6 jeux de l'analyse.  
Bien que des informations aient été trouvées sur les 4 jeux *The Elder Scrolls V: Skyrim - Dragonborn, The Elder Scrolls: Arena, Final Fantasy XVI: Echoes of the Fallen, Final Fantasy XIV: Shadowbringers*, il semble très probable qu'ils utilisent eux aussi en priorité d'autres lanceurs et ont donc également été ôtés.

In [33]:
# Elimination des jeux pour lesquels les requêtes ont échoué
df_jeux_steam = df_jeux_steam[
    (df_jeux_steam['Nom'] != 'Final Fantasy XIV Online') & 
    (df_jeux_steam['Nom'] != 'Final Fantasy XIV: Stormblood') & 
    (df_jeux_steam['Nom'] != 'The Elder Scrolls Online: Blackwood') &
    (df_jeux_steam['Nom'] != 'Final Fantasy XIV: Endwalker') & 
    (df_jeux_steam['Nom'] != 'Warhammer Online: Age of Reckoning') & 
    (df_jeux_steam['Nom'] != 'Dark Cloud 2') &
    (df_jeux_steam['Nom'] != 'The Elder Scrolls V: Skyrim - Dragonborn') & 
    (df_jeux_steam['Nom'] != 'Final Fantasy XVI: Echoes of the Fallen') & 
    (df_jeux_steam['Nom'] != 'Final Fantasy XIV: Shadowbringers') &
    (df_jeux_steam['Nom'] != 'The Elder Scrolls: Arena')
]

# Nouvelle sauvegarde des résultats
chemin_sauvegarde = 'steam_jeux_rpg.csv'
df_jeux_steam.to_csv(chemin_sauvegarde, index=False, encoding='utf-8-sig')

### Fusion des jeux de données

In [34]:
# Jointure externe droite sur les enregistrements de df_jeux_steam
df_merged = df_IGDB.merge(df_jeux_steam, on='Nom', how='right')

# Sauvegarde du dataframe global
csv_path = "jeux_rpg.csv"
df_merged.to_csv(csv_path, index=False, encoding='utf-8-sig')

display(df_merged)

,Nom,Résumé,Date de sortie,Note,Plateformes,Développeurs/Éditeurs,Modes de jeu,AppID Steam,Prix,Joueurs Actuels,Score Metacritic,Recommandations Positives,Propriétaires,Revenus Approximatifs
0,Final Fantasy XIV: Heavensward,Final Fantasy XIV: Heavensward is the first ex...,2015-06-23,86.89,"Xbox Series X|S, PlayStation 3, PlayStation 4,...","Square Enix, Square Enix Business Division 5, ...","Single player, Multiplayer, Co-operative, Mass...",39210,9.99 €,12649,83,72896,"2,000,000 .. 5,000,000","34,965,000.00 €"
1,Albion Online,Albion Online is a sandbox MMORPG set in an op...,2017-07-17,87.50,"Linux, Android, PC (Microsoft Windows), iOS, Mac",Sandbox Interactive,"Multiplayer, Massively Multiplayer Online (MMO)",761890,Gratuit,6104,N/A,1869,"2,000,000 .. 5,000,000",N/A
2,Star Wars: The Old Republic,Step in to the center of your own Star Wars st...,2011-12-20,85.00,"PC (Microsoft Windows), Mac","Electronic Arts, BioWare Austin, LucasArts, Bi...","Multiplayer, Co-operative, Massively Multiplay...",1286830,Gratuit,2095,85,410,"5,000,000 .. 10,000,000",N/A
3,Guild Wars 2,Guild Wars 2: Path of Fire is the second expan...,2017-09-22,83.75,"PC (Microsoft Windows), Mac",ArenaNet,Massively Multiplayer Online (MMO),1284210,Gratuit,2052,90,339,"2,000,000 .. 5,000,000",N/A
4,Grim Dawn: Definitive Edition,The Grim Dawn Definitive Edition includes:\n\n...,2019-12-31,85.00,"PC (Microsoft Windows), Xbox One",Crate Entertainment,"Single player, Multiplayer, Co-operative",219990,24.99 €,2037,83,83298,"10,000,000 .. 20,000,000","374,850,000.00 €"
5,Lies of P,"Inspired by the familiar story of Pinocchio, L...",2023-09-18,85.50,"Xbox Series X|S, PlayStation 4, PC (Microsoft ...","Neowiz, Round8 Studio",Single player,1627720,59.99 €,972,N/A,37216,"500,000 .. 1,000,000","44,992,500.00 €"
6,Dragon's Dogma II,"Dragon's Dogma is a single player, narrative d...",2024-03-22,90.83,"Xbox Series X|S, PC (Microsoft Windows), PlayS...",Capcom,Single player,2054970,64.99 €,838,88,69678,"2,000,000 .. 5,000,000","227,465,000.00 €"
7,Final Fantasy XVI,Final Fantasy XVI is the first fully fledged a...,2023-06-22,92.12,"PC (Microsoft Windows), PlayStation 5","Flame Hearts, Square Enix Creative Studio III,...",Single player,2515020,49.99 €,719,N/A,15263,"200,000 .. 500,000","17,496,500.00 €"
8,Kingdom Hearts Birth by Sleep,Birth by Sleep is an action roleplaying game t...,2010-01-09,82.67,PlayStation Portable,"Square Enix, Square Enix Product Development D...","Single player, Multiplayer",2552430,49.99 €,498,N/A,6257,"500,000 .. 1,000,000","37,492,500.00 €"
9,Suikoden I & II HD Remaster: Gate Rune and Dun...,A hero's destiny is written in the Stars. The ...,2025-03-06,86.60,"Xbox Series X|S, PlayStation 4, PC (Microsoft ...",Konami,Single player,1932640,49.99 €,386,82,1220,"0 .. 20,000","499,900.00 €"


**Ces 34 jeux constituent l'échantillon étudié par la suite.**